# Ecoli demo

[biomodels database](https://www.ebi.ac.uk/biomodels/)

The unit of flux: mmol/h/gDW

## flux balance analysis (FBA)

In [1]:
# import model
import cobra.test
model=cobra.test.create_test_model("ecoli")
display(model)

Name,iJO1366
Memory address,0x07f9c4574e910
Number of metabolites,1805
Number of reactions,2583
Number of groups,37
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [2]:
# search metabolites: glucose and its isomers
for x in model.metabolites:
    if x.formula == "C6H12O6":
        print(x.id,":",x.name,sep="")

all__D_c:D-Allose
fru_c:D-Fructose
gal_c:D-Galactose
glc__D_c:D-Glucose
inost_c:Myo-Inositol
man_c:D-Mannose
all__D_e:D-Allose
fru_e:D-Fructose
gal_e:D-Galactose
gal_bD_e:Beta D-Galactose
glc__D_e:D-Glucose
inost_e:Myo-Inositol
man_e:D-Mannose
all__D_p:D-Allose
fru_p:D-Fructose
gal_p:D-Galactose
gal_bD_p:Beta D-Galactose
glc__D_p:D-Glucose
inost_p:Myo-Inositol
man_p:D-Mannose


In [3]:
# PGI reaction: converts D-glucose 6-phosphate into D-fructose 6-phosphate
pgi = model.reactions.get_by_id("PGI")
display(pgi)


Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07f9c93d9b650
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b4025
Lower bound,-1000.0
Upper bound,1000.0


In [4]:
# search reactions involving glucose
glucose = model.metabolites.get_by_id("glc__D_c")
for r in model.reactions:
    if glucose in r.reactants:
        print(r.id,"|",r.name,"|",r.build_reaction_string())

GLCATr | D-glucose O-acetyltransferase | accoa_c + glc__D_c <=> acglc__D_c + coa_c
HEX1 | Hexokinase (D-glucose:ATP) | atp_c + glc__D_c --> adp_c + g6p_c + h_c
XYLI2 | Xylose isomerase | glc__D_c <=> fru_c


In [5]:
# The objective function of E.coli
model.objective.expression.__str__()

'1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1'

In [6]:
print(model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))

BIOMASS_Ec_iJO1366_core_53p95M: 0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.000122 bmocogdp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 48.601527 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.017868 pe160_c + 0.045946 pe160_p + 0.054154 pe161_c + 0.02106 pe161_p + 0.185265 phe__L_c + 0.000223 

In [7]:
# solve the model with default bounds
model.optimize()
model.summary()


In [21]:
solution = model.optimize()
solution.to_frame()

,fluxes,reduced_costs
DM_4crsol_c,0.000219,0.000000
DM_5drib_c,0.000221,0.000000
DM_aacald_c,0.000000,0.000000
DM_amob_c,0.000002,0.000000
DM_mththf_c,0.000440,0.000000
...,...,...
ZN2abcpp,0.000000,-0.008295
ZN2t3pp,0.000000,-0.002074
ZN2tpp,0.000335,0.000000
ZNabcpp,0.000000,-0.008295


## flux variability analysis (FVA)

In [8]:
# flux variability analysis for xylose isomerase
from cobra.flux_analysis import flux_variability_analysis
fv = flux_variability_analysis(model,[model.reactions.get_by_id('XYLI2')])
display(fv)

,minimum,maximum
XYLI2,-1.606050e-13,6.424199e-13


## Problems

### Change objective function
 
Calculate the solutions for the E. coli model with the following objectives:  ATP consumption; Redox exchange.  

[Changing the objectives function](https://cobrapy.readthedocs.io/en/latest/simulating.html#Changing-the-Objectives)

In [9]:
# change the objective to ATPM (single reaction)
ecoli = cobra.test.create_test_model("ecoli")
ecoli.objective = "ATPM"
ecoli.reactions.get_by_id("ATPM").upper_bound = 1000.
ecoli.optimize()
ecoli.summary()


In [10]:
# change the objective to Redox exchange (multiple reactions)
import pandas
ecoli = cobra.test.create_test_model("ecoli")
# search metabolites about redox
for x in ecoli.reactions:
    if x.name.__str__().find("redox")!=-1:
        print(x.id,":",x.name)
# define nadh as redox indicator
ecoli.metabolites.nadh_c.summary()
a=ecoli.metabolites.nadh_c.summary().to_frame()
len(a.index)
ids=[a.index[i][1] for i in range(0,13)]
# first 0~5 produce nadh, 9~13 consume nadh
produce_consume={}
for i in range(0,6):
    produce_consume[ecoli.reactions.get_by_id(ids[i])]=1.0
for i in range(6,13):
    produce_consume[ecoli.reactions.get_by_id(ids[i])]=-1.0
ecoli.objective = produce_consume
ecoli.objective.expression.__str__()
ecoli.optimize()
ecoli.summary()

GRXR : Glutaredoxin reductase
NHFRBO : NADH:flavorubredoxin oxidoreductase
PAPSR : Phosphoadenylyl-sulfate reductase (thioredoxin)
PAPSR2 : Phosphoadenylyl-sulfate reductase (glutaredoxin)
RNDR1b : Ribonucleoside-diphosphate reductase (ADP) (glutaredoxin)
RNDR2b : Ribonucleoside-diphosphate reductase (GDP) (glutaredoxin)
RNDR3b : Ribonucleoside-diphosphate reductase (CDP) (glutaredoxin)
RNDR4b : Ribonucleoside-diphosphate reductase (UDP) (glutaredoxin)
THIORDXi : Hydrogen peroxide reductase (thioredoxin)
TRDR : Thioredoxin reductase (NADPH)


### Simulate aerobic and anaerobic conditions.

In [11]:
# aerobic
ecoli = cobra.test.create_test_model("ecoli")
# o2 flux : 1000.0
print(ecoli.medium)
# return the objective value
# it's mass growth speed as default
ecoli.slim_optimize()

{'EX_ca2_e': 1000.0, 'EX_cbl1_e': 0.01, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_k_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_slnt_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_tungs_e': 1000.0, 'EX_zn2_e': 1000.0}


0.9823718127269787

In [12]:
# anaerobic
# medium change is only applied within the with block 
with ecoli:
    medium = ecoli.medium
    # can not change EX_o2_e directly
    medium["EX_o2_e"] = 0.0
    ecoli.medium = medium
    print("EX_o2_e" in ecoli.medium.keys())
    print(ecoli.slim_optimize())
# grow slower
print(ecoli.slim_optimize())
# o2 flux in original model does not change
print(ecoli.medium["EX_o2_e"])

False
0.24150155709714463
0.982371812726983
1000.0


###  Simulate gene essentiality 

[Single-Deletions](https://cobrapy.readthedocs.io/en/latest/deletions.html#Single-Deletions)  
[how-to-get-an-arbitrary-element-from-a-frozenset](https://stackoverflow.com/questions/17801665/how-to-get-an-arbitrary-element-from-a-frozenset)

In [13]:
# delete Glucose producing gene
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
# search id
metabolites=[]
for i in ecoli.metabolites:
    if i.name.__str__().find("glucose")!=-1:
        metabolites.append(i)
print(metabolites)
# search reactions
for r in ecoli.reactions:
    for i in metabolites:
        if i in r.products:
            print(r.id,"|",r.name,"|",r.build_reaction_string())

glucose=ecoli.metabolites.get_by_id("glc__D_c") 
reactions=[]
for r in ecoli.reactions:
    if glucose in r.products:
        reactions.append(r)
# 16 reactions produce glucose
len(reactions)
# delete the gene of the first reaction
gene=[i for i in reactions[0].genes][0]
# gene_list : iterable
single_gene_deletion(ecoli, [gene])


[<Metabolite acglc__D_c at 0x7f9c438f0e10>, <Metabolite adpglc_c at 0x7f9c43895e90>, <Metabolite dtdp4d6dg_c at 0x7f9c435c5d10>, <Metabolite dtdpglu_c at 0x7f9c435cd5d0>, <Metabolite udpg_c at 0x7f9c426be190>, <Metabolite udpg_e at 0x7f9c4268bd50>, <Metabolite udpg_p at 0x7f9c4261de50>]
G1PTT | Glucose-1-phosphate thymidylyltransferase | dttp_c + g1p_c + h_c --> dtdpglu_c + ppi_c
GALUi | UTP-glucose-1-phosphate uridylyltransferase (irreversible) | g1p_c + h_c + utp_c --> ppi_c + udpg_c
GLCATr | D-glucose O-acetyltransferase | accoa_c + glc__D_c <=> acglc__D_c + coa_c
GLGC | Glucose-1-phosphate adenylyltransferase | atp_c + g1p_c + h_c --> adpglc_c + ppi_c
TDPGDH | DTDPglucose 4,6-dehydratase | dtdpglu_c --> dtdp4d6dg_c + h2o_c
UDPGtex | UDPglucose transport via diffusion (extracellular to periplasm) | udpg_e <=> udpg_p


,growth,status
ids,,
(b3416),0.982372,optimal


In [14]:
# wrap in function
def findAminoProducingReaction(name):
    metabolites=[]
    for i in ecoli.metabolites:
        if i.name.__str__().find(name)!=-1:
            metabolites.append(i)
    reactions=[]
    for r in ecoli.reactions:
        for i in metabolites:
            if i in r.products:
                print(r.id,"|",r.name,"|",r.build_reaction_string())
                print(r.metabolites[i])
                reactions.append(r)
    return([metabolites,reactions])
findAminoProducingReaction("Leucine")
findAminoProducingReaction("Lysine")
findAminoProducingReaction("Methionine")
findAminoProducingReaction("Phenylalanine")
findAminoProducingReaction("Threonine")
findAminoProducingReaction("Histidine")

LEUTAi | Leucine transaminase (irreversible) | 4mop_c + glu__L_c --> akg_c + leu__L_c
1.0
LEUabcpp | L-leucine transport via ABC system (periplasm) | atp_c + h2o_c + leu__L_p --> adp_c + h_c + leu__L_c + pi_c
1.0
LEUt2rpp | L-leucine reversible transport via proton symport (periplasm) | h_p + leu__L_p <=> h_c + leu__L_c
1.0
LEUtex | L-leucine transport via diffusion (extracellular to periplasm) | leu__L_e <=> leu__L_p
1.0
CADVtpp | Lysine/Cadaverine antiporter (periplasm) | 15dap_c + h_p + lys__L_p --> 15dap_p + h_c + lys__L_c
1.0
DAPDC | Diaminopimelate decarboxylase | 26dap__M_c + h_c --> co2_c + lys__L_c
1.0
FRULYSDG | Fructoselysine phosphate deglycase | frulysp_c + h2o_c <=> g6p_c + lys__L_c
1.0
LYSTRS | Lysyl-tRNA synthetase | atp_c + lys__L_c + trnalys_c --> amp_c + lystrna_c + ppi_c
1.0
LYSabcpp | L-lysine transport via ABC system (periplasm) | atp_c + h2o_c + lys__L_p --> adp_c + h_c + lys__L_c + pi_c
1.0
LYSt2pp | L-lysine transport in via proton symport (periplasm) | h_p + l

[[<Metabolite his__L_c at 0x7f9c432aadd0>,
  <Metabolite his__L_e at 0x7f9c42718f50>,
  <Metabolite his__L_p at 0x7f9c42660850>],
 [<Reaction HISTD at 0x7f9c41f9f990>,
  <Reaction HISabcpp at 0x7f9c41fadcd0>,
  <Reaction HISt2rpp at 0x7f9c41fb2850>,
  <Reaction HIStex at 0x7f9c41fb2f90>]]

## reference

[Documentation for COBRApy](https://cobrapy.readthedocs.io/en/latest/)  

Carbonell, Pablo. Metabolic Pathway Design. Springer International Publishing, 2019.